----------2021-01-30----------------

author: shi_hailong

product: hai lingke

# 发音----->动作
### 打开空调-------->前进
### 关闭空调-------->后退
### 关闭开关-------->停止
### 打开照明-------->左转
### 关闭照明-------->右转
### 关闭电饭煲------>中间

# 0、引入库

In [1]:
from jetbot import Robot
from jetbot import Pwm

In [2]:
robot = Robot()

In [3]:
direction = Pwm()

In [4]:
#direction.switch(direction.turn_middle)
#direction.switch(direction.turn_left2)
#direction.switch(direction.turn_right2)
#robot.set_motors(0, 120)
#robot.set_motors(-100, -130)
#robot.stop()

In [5]:
import serial
import threading
import time
from IPython.display import clear_output
import ctypes
import inspect

# 1、线程的结束代码

In [6]:
def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

# 2、配置串行通信

执行下面的语句之前一定要先在终端更改/dev/ttyTHS1的权限

1、sudo chmod 777 /dev/ttyTHS1

2、输入密码

In [7]:
xiaoke = serial.Serial("/dev/ttyTHS1",115200,timeout=1)
xiaoke.xonxoff = True
xiaoke.bytesize = serial.EIGHTBITS
xiaoke.stopbits = 1

# 000、临时的存储，因为不是专用的语音识别模块，所以先用一下，到后面定制了，就不需要这几行代码了

In [8]:
should_pronounce = {'openkongtiao':'forward','closekongtiao':'backward','guanbikaiguan':'stop', 'dakaizaoming':'left','guanbizhaoming':'right','guabdianfangbao':'middle'}

#a =['openkongtiao','closekongtiao','guanbikaiguan','dakaizaoming','guanbizhaoming','guabdianfangbao']

#should_pronounce.values()
#b = should_pronounce.keys()

In [9]:
def get_control(pro_str):
    should_pronounce = {'openkongtiao':'forward','closekongtiao':'backward','guanbikaiguan':'stop', 'dakaizaoming':'left','guanbizhaoming':'right','guabdianfangbao':'middle'}
    s = should_pronounce.keys()
    for item in s:
        if item in pro_str:
            return should_pronounce.get(item)
            break
        else:
            continue
    return 'not_found'

In [10]:
#direction.switch(direction.turn_middle)
#direction.switch(direction.turn_left2)
#direction.switch(direction.turn_right2)
#robot.set_motors(0, 120)
#robot.set_motors(-100, -130)
#robot.stop()
def set_control(con_str):
    if con_str == 'forward':
        robot.set_motors(0,120)
    if con_str == 'backward':
        robot.set_motors(0,-120)
    if con_str == 'stop':
        robot.set_motors(0,0)
    if con_str == 'left':
        direction.switch(direction.turn_left2)
    if con_str == 'right':
        direction.switch(direction.turn_right2)
    if con_str == 'middle':
        direction.switch(direction.turn_middle)

# 3、解析语音识别的反馈信息

In [11]:
def getInfo():
    while True:
        while xiaoke.inWaiting()>0:
            try:
                myout=xiaoke.readline()
                clear_output(True)
                print(myout)
                s = str(myout)
                ret = get_control(s)
                set_control(ret)
                print(ret)
                #if 'open' in s:
                #    print('----open----')
            except Exception as e:
                print('exception----')
                return 1

# 4、开始线程获取语音识别结果

In [12]:

t = threading.Thread(target=getInfo)
t.setDaemon(True)
t.start()

b'uArTcP\x0e\x00\x03\xe9\x9d\xf9\x00\x07p\xe7exitUni'
not_found


# 5、结束线程

In [31]:
stop_thread(t)